In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# CSP

**Sources:**
 
 - https://www.cs.ubc.ca/~mack/CS322/lectures/3-CSP2.pdf

**Definition:** A _constraint satisfaction problem (CSP)_ consists of:

 * a set of variables $\mathscr V$.
 * a domain $\textrm{dom}(V)$ for each variable $V \in \mathscr V$.
 * a set of constraints $C$.
 
An example of a CSP model is:

 * $\mathscr V = \{V_1, V_2\}$
   * $\textrm{dom}(V_1) = \{1,2,3\}$
   * $\textrm{dom}(V_2) = \{1,2\}$
 * $C = \{C_1,C_2,C_3\}$
   * $C_1: V_2 \neq 2$
   * $C_2: V_1 + V_2 < 5$
   * $C_3: V_1 > V_2$
 
**Definition**: A _model_ of a CSP is an assignment of values to all of its variables that _satisfies_ all of its constraints.

**Generate and Test (GT) algorithm**: Systematically check all possible worlds. All possible worlds is the cross product of all the domains:

$$ \textrm{dom}(V_1) \times \textrm{dom}(V_2) \times \ldots \times \textrm{dom}(V_n) $$

Generate and test:

 1. Generate possible worlds one at a time.
 2. Test constraints for each one.
 
For $k$ variables, each with domain size $d$, and there are $c$ constraints, the complexity is $O(ck^d)$:

 * There are $d^k$ possible worlds.
 * For each one need to check c constraints.
 
**Graph Searching (GS) algorithm (backtrack)**: In this algorithm we have a set of _states_, which are partial assignments of values to variables. At the start state there are no assignments. A successor function is defined with states with the next variable assigned:

 * E.g., follow a total order of the variables $\{V_1,\ldots,V_n\}$.
 * A state assigns values to the first $k$ variables $\{V_1,\ldots,V_n\}$
   * Neighbors of node $\{V_1 = v_1, \ldots, V_k=v_k, V_{k+1}=x\}$ for each $x \in \textrm{dom}(V_{k+1})$.
   
Goal state: complete assignments of values to variables that satisfy all constraints. Solution is the assignment (path does not matter).

The search space can be explored via DFS but evaluate each constraint as soon as all its variables are bound (all variables in a constrant, are in scope). Any partial assignment that doesn't satisfy the constraint can be pruned.

A heuristic can be added to this algorithm. An order can be applied to the checking of the variables. It should start to check the variable that is used in the most constraints first (fail early).

# Basic algorithm implementation (`HSSolver`)

In [23]:
import copy

def head(xs): return xs[0] if len(xs) > 0 else None
def replace_all(text, dic):
    for k, v in dic.items():
        text = text.replace(k, str(v))
    return text
def subset(A,B): return all([a in B for a in A])

class Variable():
    def __init__(self, name, D):
        self.name = name
        self.domain = D
        self.value = D[0]
    
    def __eq__(self, v): return v.name == self.name
    def __repr__(self): return '{}={}'.format(self.name, self.value, self.domain)
        
class Constraint():
    def __init__(self, expr, *variables):
        self.variables = variables
        self.expression = expr
    def evalstr(self): return 
    def eval(self, V): 
        return eval(replace_all(self.expression, {v.name:v.value for v in V}))
    def __repr__(self): return self.expression
    
class CSP():
    def __init__(self, variables, constraints):
        self.variables = variables
        self.constraints = constraints
        self.stop_on_first = False
        
    def solve(self):
        self.solution = []
        self.gt_solve([], self.variables)
        return self.solution[::-1]
    
    # solving with a Generate and Test (GT) algorithm
    #
    # example:
    # for a \in dom A
    #   for b \in dom B
    #      for c \in dom C
    #        if {A=a, B=b, C=c} return solution
    def gt_solve(self, S, V):
        print('(Call) S contains {}, V contains {}'.format(S,V))
        if len(V) == 0:
            
            # -- eval all
            #return all([c.eval(S) for c in self.constraints])
            
            # -- verbose output
            print('  (Base) Checking constraints...')
            for c in self.constraints:
                r = c.eval(S)
                if r: print('  (Satisfied) {}'.format(c))
                else: 
                    print('  (Failed) {}'.format(c))
                    return False
            return True
            
        v = V.pop()
        S.append(v)
        for d in v.domain:
            if self.solution and self.stop_on_first: return
            v.value = d
            if self.gt_solve(copy.deepcopy(S), copy.deepcopy(V)):
                self.solution = S
                print('  (Solution) {}'.format(S[::-1]))
                return
        return False

## Example 1

Testing the `gt_solve` method with the example model shown at the beginning of this document:

In [476]:
v1 = Variable('v1', [1,2,3])
v2 = Variable('v2', [1,2])
V = [v1,v2]
c1 = Constraint('v1 != v2', v1, v2)
c2 = Constraint('v1 + v2 <= 5', v1, v2)
c3 = Constraint('v1 > v2', v1, v2)
c4 = Constraint('v1 >= 3', v1)
C = [c1,c2,c3,c4]   
csp = CSP(V, C)
csp.stop_on_first = True
csp.solve()

(Call) S contains [], V contains [v1=1, v2=1]
(Call) S contains [v2=1], V contains [v1=1]
(Call) S contains [v2=1, v1=1], V contains []
  (Base) Checking constraints...
  (Failed) v1 != v2
(Call) S contains [v2=1, v1=2], V contains []
  (Base) Checking constraints...
  (Satisfied) v1 != v2
  (Satisfied) v1 + v2 <= 5
  (Satisfied) v1 > v2
  (Failed) v1 >= 3
(Call) S contains [v2=1, v1=3], V contains []
  (Base) Checking constraints...
  (Satisfied) v1 != v2
  (Satisfied) v1 + v2 <= 5
  (Satisfied) v1 > v2
  (Satisfied) v1 >= 3
  (Solution) [v1=3, v2=1]


[v1=3, v2=1]

## Example 2

Solving the Australian map coloring problem.

In [471]:
%%time
colors = {'red': 0, 'blue': 1, 'green': 2}
icolors = {v:k for k,v in colors.items()}
V = [Variable(x, list(colors.values())) for x in 'SA,WA,NT,Q,NSW,V'.split(',')]
V
c1 = Constraint('SA!=WA')
c2 = Constraint('SA!=NT')
c3 = Constraint('SA!=Q')
c4 = Constraint('SA!=NSW')
c5 = Constraint('SA!=V')
c6 = Constraint('WA!=NT')
c7 = Constraint('NT!=Q')
c8 = Constraint('Q!=NSW')
c9 = Constraint('NSW!=V')
C = [c1,c2,c3,c4,c5,c6,c7,c8,c9]
csp = CSP(V,C)
csp.stop_on_first = True
csp.solve()

(Call) S contains [], V contains [SA=0, WA=0, NT=0, Q=0, NSW=0, V=0]
(Call) S contains [V=0], V contains [SA=0, WA=0, NT=0, Q=0, NSW=0]
(Call) S contains [V=0, NSW=0], V contains [SA=0, WA=0, NT=0, Q=0]
(Call) S contains [V=0, NSW=0, Q=0], V contains [SA=0, WA=0, NT=0]
(Call) S contains [V=0, NSW=0, Q=0, NT=0], V contains [SA=0, WA=0]
(Call) S contains [V=0, NSW=0, Q=0, NT=0, WA=0], V contains [SA=0]
(Call) S contains [V=0, NSW=0, Q=0, NT=0, WA=0, SA=0], V contains []
  (Base) Checking constraints...
  (Failed) SA!=WA
(Call) S contains [V=0, NSW=0, Q=0, NT=0, WA=0, SA=1], V contains []
  (Base) Checking constraints...
  (Satisfied) SA!=WA
  (Satisfied) SA!=NT
  (Satisfied) SA!=Q
  (Satisfied) SA!=NSW
  (Satisfied) SA!=V
  (Failed) WA!=NT
(Call) S contains [V=0, NSW=0, Q=0, NT=0, WA=0, SA=2], V contains []
  (Base) Checking constraints...
  (Satisfied) SA!=WA
  (Satisfied) SA!=NT
  (Satisfied) SA!=Q
  (Satisfied) SA!=NSW
  (Satisfied) SA!=V
  (Failed) WA!=NT
(Call) S contains [V=0, NSW=

In [468]:
[{v.name:icolors[v.value]} for v in csp.solution]

[{'V': 'red'},
 {'NSW': 'blue'},
 {'Q': 'red'},
 {'NT': 'blue'},
 {'WA': 'red'},
 {'SA': 'green'}]

# Implementations of all algorithms

This section covers all the implementations of the algorithms. In later sections, each algorithm is explained in more detail. The following algorithms are implemented:

 * `HSSolver`: brute-force tree-based search. (brute-force)
 * `GSSolver`: pruned tree-based search. (evaluate constraints when variables are in scope)
 * `HGSSolver`: heuristical pruned tree-based search. (solve the variables in order of max. number of constraints)
 * `IGSSolver`: inferenced pruned tree-based search. (remove any assigned values from the domain of any shared $\mathrm{AllDiff}$s)

In [177]:
import copy

class Variable():
    def __init__(self, name, D): 
        self.name = name; 
        self.domain = D; 
        self.value = D[0]
    def __eq__(self, v): return v.name == self.name
    def __repr__(self): return '{}={}'.format(self.name, self.value, self.domain)
        
class Constraint():
    def __init__(self, expr, scope): self.expression = expr; self.scope = scope
    def __repr__(self): return self.expression
    def eval(self, V): return eval(self.replace_all(self.expression, {v.name:v.value for v in V}))
    def replace_all(self, text, dic):
        for k, v in dic.items():
            text = text.replace(k, str(v))
        return text
    
class AllDiff(Constraint):
    def __init__(self, scope): self.scope = scope
    def eval(self, V): return eval(self.replace_all('alldiff([{}])'
                                                    .format(','.join([v.name for v in V])), {v.name:v.value for v in V}))

# Scopeless brute-force search algorithm.
class GTSolver():
   
    def __init__(self, variables, constraints):
        self.variables = variables
        self.constraints = constraints
        self.stop_on_first = False
        
    def solve(self):
        self.solution = []
        self.gt_solve([], self.variables)
        return self.solution[::-1]
    
    def gt_solve(self, S, V):
        if len(V) == 0: 
            return all([c.eval(S) for c in self.constraints])           
        v = V.pop()
        S.append(v)
        for d in v.domain:
            if self.solution and self.stop_on_first: return
            v.value = d
            if self.gt_solve(copy.deepcopy(S), copy.deepcopy(V)):
                self.solution.append(S)
        return False
    
# Scope searching algorithm with solution pruning. It starts
# to check if a solution is valid as soon as the scope of a constraint
# is a subset of the solution set.
class GSSolver():
    
    def all_scope(self, C, S): return all([s in C for s in S])
    def any_scope(self, C, S): return any([s in C for s in S])
    def k_scope(self, C, S, k): return sum([1 for s in S if s in C]) >= k
    
    def __init__(self, variables, constraints):
        self.variables = variables
        self.constraints = constraints
        self.stop_on_first = False
        
    def solve(self):
        self.solution = []
        self.gs_solve([], self.variables)
        return self.solution[::-1]
    
    def gs_solve(self, S, V):
        if len(V) == 0: 
            return all([c.eval(S) for c in self.constraints]) 
        elif len(S) > 0:
            for c in self.constraints:
                if isinstance(c, AllDiff) and self.k_scope(S, c.scope, 2):
                    if not c.eval(S): return
                else:
                    if self.all_scope(S, c.scope) and not c.eval(S):
                        return
        v = V.pop()
        S.append(v)
        for d in v.domain:
            if self.solution and self.stop_on_first: return
            v.value = d
            if self.gs_solve(copy.deepcopy(S), copy.deepcopy(V)):
                self.solution.append(S)
        return False
    
# Heuristical scope searching algorithm with solution pruning. It starts
# to check if a solution is valid as soon as the scope of a constraint
# is a subset of the solution set. Before solving, by inference, it will sort
# the variables based on in how many constraints they are. With this heuristic
# it will fail as early as possible (which should improve the speed).
class HGSSolver():
    
    # C : scope of the constraint
    # S : solution set
    def all_scope(self, C, S): return all([s in C for s in S]) # c is a subset of S
    def any_scope(self, C, S): return any([s in C for s in S]) # any c is in S
    def k_scope(self, C, S, k): return sum([1 for s in S if s in C]) >= k # c is in S atleast k times
    
    def __init__(self, variables, constraints):
        self.variables = variables
        self.constraints = constraints
        self.stop_on_first = False
        
    def solve(self):
        self.solution = []
        self.gs_solve([], self.variables)
        return self.solution[::-1]
    
    def gs_solve(self, S, V):
        if len(V) == 0: 
            return all([c.eval(S) for c in self.constraints]) 
        elif len(S) > 0:
            for c in self.constraints:
                if isinstance(c, AllDiff) and self.k_scope(S, c.scope, 2): 
                    if not c.eval(S): return
                else:
                    if self.all_scope(S, c.scope) and not c.eval(S): return
        v = V.pop()
        S.append(v)
        for d in v.domain:
            if self.solution and self.stop_on_first: return
            v.value = d
            if self.gs_solve(copy.deepcopy(S), copy.deepcopy(V)):
                self.solution.append(S)
        return False

Applying this to the map coloring problem:

In [178]:
def color_problem():
    colors = {'red': 0, 'blue': 1, 'green': 2}
    V = [Variable(x, list(colors.values())) for x in 'SA,WA,NT,Q,NSW,V'.split(',')]
    def f(name): return head(list(filter(lambda v: v.name == name, V)))
    c1 = Constraint('SA!=WA', [f('SA'), f('WA')])
    c2 = Constraint('SA!=NT', [f('SA'), f('NT')])
    c3 = Constraint('SA!=Q', [f('SA'), f('Q')])
    c4 = Constraint('SA!=NSW', [f('SA'), f('NSW')])
    c5 = Constraint('SA!=V', [f('SA'), f('V')])
    c6 = Constraint('WA!=NT', [f('WA'), f('NT')])
    c7 = Constraint('NT!=Q', [f('NT'), f('Q')])
    c8 = Constraint('Q!=NSW', [f('Q'), f('NSW')])
    c9 = Constraint('NSW!=V', [f('NSW'), f('V')])
    C = [c1,c2,c3,c4,c5,c6,c7,c8,c9]
    return (V, C)

In [179]:
%%time
V, C = color_problem()
csp = GTSolver(V,C)
csp.solve()
print('There are {} solutions:'.format(len(csp.solution)))
lkup = {v:k for k,v in colors.items()}
print([[{v.name:lkup[v.value]} for v in s] for s in csp.solution])

There are 6 solutions:
[[{'V': 'red'}, {'NSW': 'blue'}, {'Q': 'red'}, {'NT': 'blue'}, {'WA': 'red'}, {'SA': 'green'}], [{'V': 'red'}, {'NSW': 'green'}, {'Q': 'red'}, {'NT': 'green'}, {'WA': 'red'}, {'SA': 'green'}], [{'V': 'blue'}, {'NSW': 'red'}, {'Q': 'blue'}, {'NT': 'red'}, {'WA': 'blue'}, {'SA': 'green'}], [{'V': 'blue'}, {'NSW': 'green'}, {'Q': 'blue'}, {'NT': 'green'}, {'WA': 'blue'}, {'SA': 'green'}], [{'V': 'green'}, {'NSW': 'red'}, {'Q': 'green'}, {'NT': 'red'}, {'WA': 'green'}, {'SA': 'green'}], [{'V': 'green'}, {'NSW': 'blue'}, {'Q': 'green'}, {'NT': 'blue'}, {'WA': 'green'}, {'SA': 'green'}]]
Wall time: 190 ms


The map coloring problem contains $O(ck^d)=O(9\cdot 7^3)=3,087$ worlds.

# AllDiff and pruning (`GSSolver`)

The $\textrm{AllDiff}$ function returns true if all the elements in $S$ are different, and otherwise false. This is a useful function to set up constraints, i.e.: a Sudoku solver.

In [172]:
def alldiff(S): return len(S) == len(set(S))

**Solving with the GT algorithm**: If we create a simple problem with variables $\{V_1,\ldots,V_9\}$, with each a domain of $\textrm{dom}(V_k)=\{0,1,2,\ldots,9\}$. The constraint is that all variables have different values. The GT-algorithm runs with $O(ck^d)$ complexity, which gives us $O(1\cdot 9^9)=387,420,489$ possible worlds. By the way the algorithm works, the first solution will be $\{V_1=9, V_2=8,\ldots,V_8=2,V_9=1\}$, which is found halfway through all the worlds. This means that a solution is approximately found at the $193,710,224$th test. Needlessly to say, this algorithm will take a very long time to find a feasible solution to the model.

**Extending to the GS algorithm:** To solve this particular problem, the GT algorithm has been modified to do the GS algorithm. To properly perform this, a scope has been added to the constraints. This scope is used to start early evaluation of possible solutions. When the variables of the scope of a constraint is a subset of the solution set, we can see if the constraint is satisfied. This way, when there is an invalid solution, the sub-tree will be pruned as soon as any constraint fails to be satisfied.

The constraints will need to know which variables are included, which is the _scope_ of the constraint. If all the variables that are in the solution cover the scope of a constraint, it will be evaluated.

In [173]:
class AllDiff(Constraint):
    def __init__(self, scope): self.scope = scope
    def eval(self, V): return eval(self.replace_all('alldiff([{}])'
                                                    .format(','.join([v.name for v in V])), {v.name:v.value for v in V}))

The $\textrm{AllDiff}$ constraint does not require to have all the variables in scope. It should check satisfaction as soon as there are two or more variables in the scope that can be tested. In essence, the $\textrm{AllDiff}$ constraint will create a cross-product between all the variables to create a constraint for inequality between any 2-pairs of variables. Therefore it will create $k^2$ constraints, and we can already start checking if we have two variables in scope.

In [183]:
%%time
V = [Variable('x{}'.format(x), list(range(10))) for x in range(10)]
csp = GSSolver(V, [AllDiff(V)])
csp.stop_on_first = True
csp.solve()
print(head(csp.solution))

[x9=0, x8=1, x7=2, x6=3, x5=4, x4=5, x3=6, x2=7, x1=8, x0=9]
Wall time: 20 ms


If we create a very large problem with $k=100$, and $D_k=\{0,\ldots,99\}$, the complexity of this problem is $O(ck^d) = O(100^{100})$. The $\mathrm{AllDiff}$ constraint really shines here. To create inequalities for all $100$ variables, we would get $100^2=10,000$ different constraints. The complexity would have been $O(10,000\cdot100^{100})$, which is just $O(100^{102})$, quite an improvement. Also considering that it doesn't have to iterate over $10,000$ constraints, and check them one by one. 

In [174]:
%%time
V = [Variable('x{}'.format(x), list(range(100))) for x in range(100)]
csp = GSSolver(V, [AllDiff(V)])
csp.stop_on_first = True
csp.solve()
print(head(csp.solution))

[x99=0, x98=1, x97=2, x96=3, x95=4, x94=5, x93=6, x92=7, x91=8, x90=9, x89=10, x88=11, x87=12, x86=13, x85=14, x84=15, x83=16, x82=17, x81=18, x80=19, x79=20, x78=21, x77=22, x76=23, x75=24, x74=25, x73=26, x72=27, x71=28, x70=29, x69=30, x68=31, x67=32, x66=33, x65=34, x64=35, x63=36, x62=37, x61=38, x60=39, x59=40, x58=41, x57=42, x56=43, x55=44, x54=45, x53=46, x52=47, x51=48, x50=49, x49=50, x48=51, x47=52, x46=53, x45=54, x44=55, x43=56, x42=57, x41=58, x40=59, x39=60, x38=61, x37=62, x36=63, x35=64, x34=65, x33=66, x32=67, x31=68, x30=69, x29=70, x28=71, x27=72, x26=73, x25=74, x24=75, x23=76, x22=77, x21=78, x20=79, x19=80, x18=81, x17=82, x16=83, x15=84, x14=85, x13=86, x12=87, x11=88, x10=89, x9=90, x8=91, x7=92, x6=93, x5=94, x4=95, x3=96, x2=97, x1=98, x0=99]
Wall time: 45.6 s


It still finds a solution within a minute.

# Search heuristic (`HGSSolver`)

Let's start with applying the GS algorithm to the map coloring problem:

In [167]:
%%time
V, C = color_problem()
csp = GSSolver(V,C)
csp.solve()
print('There are {} solutions:'.format(len(csp.solution)))
lkup = {v:k for k,v in colors.items()}
print([[{v.name:lkup[v.value]} for v in s] for s in csp.solution])

There are 6 solutions:
[[{'V': 'red'}, {'NSW': 'blue'}, {'Q': 'red'}, {'NT': 'blue'}, {'WA': 'red'}, {'SA': 'green'}], [{'V': 'red'}, {'NSW': 'green'}, {'Q': 'red'}, {'NT': 'green'}, {'WA': 'red'}, {'SA': 'green'}], [{'V': 'blue'}, {'NSW': 'red'}, {'Q': 'blue'}, {'NT': 'red'}, {'WA': 'blue'}, {'SA': 'green'}], [{'V': 'blue'}, {'NSW': 'green'}, {'Q': 'blue'}, {'NT': 'green'}, {'WA': 'blue'}, {'SA': 'green'}], [{'V': 'green'}, {'NSW': 'red'}, {'Q': 'green'}, {'NT': 'red'}, {'WA': 'green'}, {'SA': 'green'}], [{'V': 'green'}, {'NSW': 'blue'}, {'Q': 'green'}, {'NT': 'blue'}, {'WA': 'green'}, {'SA': 'green'}]]
Wall time: 53 ms


It is faster than the GT algorithm, but it can be improved further. If it starts with the assignment of variables that are ordered by in how many constraints they are used, it will fail early. For the $\textrm{AllDiff}$ where all the variables share an equal amount of constraints, this will have no effect. However, for the map coloring problem this should give a speed improvement.

**Pre-sort algorithm:**

 * For each $V \in \mathscr{V}$:
   * Count in how many constraints $V$ is used.
 * Sort all the variables based on the count (large to small).
 * Call the `hgs_solve(...)` function with the sorted variables in $\mathscr{V}$.

In [181]:
%%time
V, C = color_problem()
csp = HGSSolver(V,C)
csp.solve()
print('There are {} solutions:'.format(len(csp.solution)))
lkup = {v:k for k,v in colors.items()}
print([[{v.name:lkup[v.value]} for v in s] for s in csp.solution])

There are 6 solutions:
[[{'V': 'red'}, {'NSW': 'blue'}, {'Q': 'red'}, {'NT': 'blue'}, {'WA': 'red'}, {'SA': 'green'}], [{'V': 'red'}, {'NSW': 'green'}, {'Q': 'red'}, {'NT': 'green'}, {'WA': 'red'}, {'SA': 'green'}], [{'V': 'blue'}, {'NSW': 'red'}, {'Q': 'blue'}, {'NT': 'red'}, {'WA': 'blue'}, {'SA': 'green'}], [{'V': 'blue'}, {'NSW': 'green'}, {'Q': 'blue'}, {'NT': 'green'}, {'WA': 'blue'}, {'SA': 'green'}], [{'V': 'green'}, {'NSW': 'red'}, {'Q': 'green'}, {'NT': 'red'}, {'WA': 'green'}, {'SA': 'green'}], [{'V': 'green'}, {'NSW': 'blue'}, {'Q': 'green'}, {'NT': 'blue'}, {'WA': 'green'}, {'SA': 'green'}]]
Wall time: 55 ms


# Inference (`IGSSolver`)

## Cycling

Now how can we speed up the $O(100^{100})$ complexity problem? We know that each variable should be different. Therefore can infer that we shouldn't use the value of the previous variable. Instead, it should cycle through the domain, whilst assigning variables. If it starts at $0$ and cycles through to $99$, the first generated world is also a solution, massive speed up!

But variables can contain different domains, so we can only cycle through the domain of the variable, if the previously assigned variable has the same domain.

Therefore we can state that if the previous assigned variable domain is equal to the current one, we can increment the value with modulo $\max \{\textrm{dom}(V_k)\}$. To do this, the loop where we iterate over all the domains, should shift it values (wrapping around) such that the increment is the first element in the list.

A more sophisticated and generic approach is described in the next section.

## Domain pruning

If any $\mathrm{AllDiff}$ constraints are used, we can infer some information about the domains of other variables. Let the variables be $\mathscr{V}=\{V_1,V_2,V_3,V_4,V_5\}$ with domain $\textrm{dom}(V_k) = \{1,\ldots,k\}$ where $k=5$. Under the constraint that $\textrm{AllDiff}(V_2,V_3,V_4)$. Assume $V_2=1$. By inference, we know that $V_3$ and $V_4$ cannot equal $V_2=1$. Therefore, when assigning $V_2$, the algorithm should check in which $\textrm{AllDiff}$s the variable is used, and remove the assigned value from the domain of the other variables that are in the $\textrm{AllDiff}$.

**Domain pruning algorithm:** Let $\mathscr{V}=\{V_1,V_2,\ldots,V_k\}$ with $\textrm{dom}(V_k)=\{0,1,2,\ldots,k-1\}$ where all the variables share the constraint $\textrm{AllDiff}(V_1,V_2,\ldots,V_k)$.

 1. Start with $k=0$.
 2. Take the first value $v$ from $\textrm{dom}(V_k)$.
   1. If $\textrm{dom}(V_k) = \emptyset$ then set $V_k=v$ and goto 6. 
 3. Set the variable $V_k=v$.
 4. Remove $v$ from the domain of all variables $V$ that come after $V_k$.
 5. Increment $k$ by one.
 6. For any other variables, goto 2.
 
This will result in the following domains for the first world:

$$ \begin{align} \textrm{dom}(V_1) &= \{0,1,2,\ldots,k-1\} \\ \textrm{dom}(V_2) &= \{1,2,\ldots,k-1\} \\ \textrm{dom}(V_3) &= \{2,\ldots,k-1\} \\ &\vdots \\ \textrm{dom}(V_k) &= \{k-1\} \end{align}$$

It might be obvious, but the important part is that this algorithm will always pick a different value than the previous assigned value. The algorithm assumes that all the variables in the $\textrm{AllDiff}$ have an equal domain. The first world is also a solution to the $O(100^{100})$ problem, which means that we achieve the same behaviour as described in previous section (Cycling).

# Applications

## Sudoku solver

## N-Queens problem